<a href="https://colab.research.google.com/github/venki24/Brain-Tumor-Using-CNN-/blob/master/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras 
from keras.models import Sequential
from keras.layers import Input,Add,Dense, Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D,AveragePooling2D,MaxPooling2D
from keras.models import Model,load_model
from keras.initializers import glorot_uniform
import keras.backend as K
import tensorflow as tf
import cv2
import os
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ROWS = 32
COLS = 32
CHANNELS = 3
CLASSES = 5

In [0]:
def Data_prep():
    
    train_gen = image.ImageDataGenerator(rescale = 1./255,
                                    featurewise_center=True, 
                                    samplewise_center=True, 
                                    featurewise_std_normalization=True, 
                                    samplewise_std_normalization=True, 
                                    zca_whitening=False, zca_epsilon=1e-06, 
                                    rotation_range=40, width_shift_range=0.0,
                                    height_shift_range=0.0, brightness_range=(0.2, 0.2),
                                    shear_range=0.2, zoom_range=0.2, 
                                    channel_shift_range=0.0, fill_mode='nearest', 
                                    cval=0.0, horizontal_flip=True, vertical_flip=True,
                                    data_format=None, validation_split=0.0, dtype=None)
    
    test_gen = image.ImageDataGenerator(rescale = 1./255,
                                    featurewise_center=True, 
                                    samplewise_center=True, 
                                    featurewise_std_normalization=True, 
                                    samplewise_std_normalization=True, 
                                    zca_whitening=False, zca_epsilon=1e-06, 
                                    rotation_range=40, width_shift_range=0.0,
                                    height_shift_range=0.0, brightness_range=(0.2, 0.2),
                                    shear_range=0.2, zoom_range=0.2, 
                                    channel_shift_range=0.0, fill_mode='nearest', 
                                    cval=0.0, horizontal_flip=True, vertical_flip=True,
                                    data_format=None, validation_split=0.0, dtype=None)
    
    val_gen = image.ImageDataGenerator(rescale = 1./255,
                                    featurewise_center=True, 
                                    samplewise_center=True, 
                                    featurewise_std_normalization=True, 
                                    samplewise_std_normalization=True, 
                                    zca_whitening=False, zca_epsilon=1e-06, 
                                    rotation_range=40, width_shift_range=0.0,
                                    height_shift_range=0.0, brightness_range=(0.2, 0.2),
                                    shear_range=0.2, zoom_range=0.2, 
                                    channel_shift_range=0.0, fill_mode='nearest', 
                                    cval=0.0, horizontal_flip=True, vertical_flip=True,
                                    data_format=None, validation_split=0.0, dtype=None)
    
    
    train_dir = train_gen.flow_from_directory(r"/content/drive/My Drive/Capdata/train",
                                           target_size = (32, 32),
                                           classes = ['Bunny', 'Mahesh Babu', 'Rana', 'Venkatesh','YS Jagan Mohan Reddy'],
                                           class_mode = 'categorical',
                                           batch_size = 64,
                                           seed = 1)
    
    test_dir = test_gen.flow_from_directory(r"/content/drive/My Drive/Capdata/test",
                                           target_size = (32, 32),
                                           classes = ['Bunny', 'Mahesh Babu', 'Rana', 'Venkatesh','YS Jagan Mohan Reddy'],
                                           class_mode = 'categorical',
                                           batch_size = 64,
                                           seed = 1)
    
    val_dir = val_gen.flow_from_directory(r"/content/drive/My Drive/Capdata/val",
                                           target_size = (32, 32),
                                           classes = ['Bunny', 'Mahesh Babu', 'Rana', 'Venkatesh','YS Jagan Mohan Reddy'],
                                           class_mode = 'categorical',
                                           batch_size = 64,
                                           seed = 1)
    
    return train_dir, test_dir, val_dir

In [0]:
def identity_block(X,f,filters,stage,block):
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  F1,F2,F3=filters

  X_shortcut=X

  X=Conv2D(filters=F1,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2a',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
  X=Activation('relu')(X)

  X=Conv2D(filters=F2,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2b',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
  X=Activation('relu')(X)


  X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)
  X=Activation('relu')(X)
  X=Add()([X,X_shortcut])
  X=Activation('relu')(X)

  return X



In [6]:
tf.reset_default_graph()
with tf.Session() as test:
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = ", out[0][1][1][0])









out =  [0.         0.         0.31625795 0.         0.96211    0.        ]


In [0]:
def convolutional_block(X,f,filters,stage,block,s=2):
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  F1,F2,F3=filters
  X_shortcut=X

  X=Conv2D(F1,(1,1),strides=(s,s),name=conv_name_base +'2a',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base +'2a')(X)
  X=Activation('relu')(X)

  X=Conv2D(F2,(1,1),strides=(s,s),name=conv_name_base +'2b',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base +'2b')(X)
  X=Activation('relu')(X)


  X=Conv2D(F3,(1,1),strides=(s,s),name=conv_name_base +'2c',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base +'2c')(X)
  X=Activation('relu')(X)


  X_shortcut=Conv2D(F3,(1,1),strides=(s,s),name=conv_name_base+'1',kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
  X_shortcut=BatchNormalization(axis=3,name = bn_name_base + '1')(X_shortcut)


  X=Add()([X,X_shortcut])
  X=Activation('relu')(X)

  return X

In [0]:
def ResNet50(input_shape = (32,32,3),classes=5):
  X_input=Input(input_shape)
  X=ZeroPadding2D((3,3))(X_input)

  

  X=Conv2D(64,(8,8),strides=(2,2),name='conv1',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name='bn_cnv1')(X)
  X=MaxPooling2D((3,3),strides=(2,2))(X)



  X=convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
  X=identity_block(X,3,[64,64,256],stage=2,block='b')
  X=identity_block(X,3,[64,64,256],stage=2,block='c')

  
  X=convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
  X=identity_block(X,3,[128,128,512],stage=2,block='b')
  X=identity_block(X,3,[128,128,512],stage=2,block='c')
  X=identity_block(X,3,[128,128,512],stage=2,block='d')

  X=convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
  X=identity_block(X,3,[256,256,1024],stage=2,block='b')
  X=identity_block(X,3,[256,256,1024],stage=2,block='c')
  X=identity_block(X,3,[256,256,1024],stage=2,block='d')
  X=identity_block(X,3,[256,256,1024],stage=2,block='e')
  X=identity_block(X,3,[256,256,1024],stage=2,block='f')


  X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

  X = AveragePooling2D((2, 2), name='avg_pool')(X)


  X=Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
  model= Model(inputs = X_input, outputs = X, name='ResNet50')
  
  return model



In [43]:
model = ResNet50((32,32,3), 5)

ValueError: ignored